In [16]:
import pandas as pd 
csv = pd.read_csv('f2i_with_age.csv')
csv.head(10)

,age,sex,clear_finding,impression
0,37,女,青年。女。两肺肺纹理增多、增粗。,支气管炎改变，请结合临床。
1,83,男,老年。男。肋间隙增宽；两肺肺纹理增多、增粗、紊乱、模糊；左肺中野、左肺下野、右肺中野、右肺下...,慢性支气管炎伴感染；肺气肿；建议结合临床治疗后复查。
2,55,女,中年。女。两肺肺纹理增粗、紊乱、模糊。,支气管炎 征象。清结合临床。
3,57,女,中年。女。右侧第八肋骨骨质不连续。,右侧第8肋骨骨折复查
4,73,男,老年。男。胸廓欠对称；气管欠居中；两肺肺纹理增多、增粗、模糊；右肺上野肺野透亮度增高，左肺中...,1、慢性支气管炎、肺气肿伴感染征象，请结合临床。\r\n2、主动脉粥样硬化。
5,61,男,中年。男。两肺肺纹理增多、增粗。,两肺未见明显实质性病变，请结合临床。
6,90,女,老年。女。两肺肺纹理增多、增粗、紊乱；两侧肺门增浓；两侧膈面欠光滑。,慢性支气管疾患。
7,72,女,老年。女。主动脉迂曲，主动脉增宽，主动脉球部钙化。,主动脉弓壁钙化，随访。
8,70,男,老年。男。左肺上野见斑片状高密度影。,左上肺钙化灶。
9,83,男,老年。男。右肺下野见斑片状高密度影，左肺下野、右肺下野见空洞影，呈薄壁样改变；左侧肋膈角变钝。,考虑右下肺感染，两下肺大疱，左侧胸膜肥厚，请结合临床。


In [17]:
import re

def clear_impression(impression):
        sent_cleaner = lambda t: re.sub(
            '；+',  # 清洗多个连续分号
            '；',
            re.sub('\d(?:、|\.|：|:|;|，)(?=\D)',  # 匹配标题符，匹配(1、 1: 1.)后向界定为非数字模式
                   '；',
                   re.sub(
                       '[。?？;!！（）()]',  # 匹配非法字符
                       '；',
                       t.lower().strip().replace('"', '').replace('/', '、').replace('\\', '').replace("'", '').
                           replace("\r\n", '；').replace(' ', '').replace(',', '，').replace('\n', '；').replace('\r', '').replace(' ', '').
                           replace('————', '').replace('--', '-').replace('~', '-')
                   )))
        impression = sent_cleaner(impression)
        # 清洗无用日期
        res = re.search('[0-9]{4}-\d+-\d+', impression)
        if res:
            impression = impression.replace(res.group(), '前')
            
        # 清洗 3-6 为 3、4、5、6
        res = (re.search('\d+-\d+', impression))
        if res:
            target = res.group()
            impression = impression.replace(target, '、'.join([str(i) for i in range(int(target[:target.index('-')]), int(target[target.index('-')+1:])+1)]))
        impression = re.sub('；+', '；',impression)
        
        # Python strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
        impression = impression.strip('、').strip('，').strip('；').strip('.')
        impression += '。'
        # print(repr(impression)) # 测试输出
        # time.sleep(0.2)
    
        return impression

In [18]:
from tqdm import tqdm
data_clear = [clear_impression(row['impression']) for index, row in tqdm(csv.iterrows())]

csv['impression'] = data_clear

112905it [00:03, 31298.04it/s]


In [19]:
csv = csv[['age', 'sex', 'clear_finding', 'impression']]

In [20]:
csv.to_csv('f2i_with_age_clean.csv', encoding='utf-8', index=None)